# Methods

- We have the discretized CRSS dataset in '../../Big_Files/Discretized_All_12_22_22.csv'
- MissForest is a round-robin imputation method implemented in R, generally considered one of the best imputation methods.  It has several Python implementations.
- I tried to use MissForest, https://pypi.org/project/MissForest/, to impute missing values, but it gave me errors, and finding the source of the errors led me down the path to write my own round-robin implementation.
- I compare here three methods:
    - Round-Robin Random Forest (my own implementation of Round Robin, using scikit-learn's random forest)
    - Imputation by mode
    - IVEware, using the hyperparameters in the CRSS Imputation report
- To compare, I followed the example for MissForest.
    - I dropped all samples with a missing value, so I would have ground truth.
    - I erased 15% of the values in each sample.
    - I used each imputation method to impute the missing values, and, for each feature, counted how many did not match the ground truth.
- My round-robin method
    - In data_NaN, change all of the 'Unknown' to np.NaN.
    - In each feature, count the number of unknown samples.
    - In another copy, data_Mode, impute by mode in all of the features.
    - Starting with the feature with the least (nonzero) number of missing samples:
        - Copy that feature from data_NaN into data_Mode, so that only that feature has missing values.
        - Separate the dataframe into two, one with known values in the target variable (X) and one with unknown values (Z).
        - From the dataframe with known values (X), separate out the target variable (call it 'y')
        - Using Random Forest, build a model that maps X to y.  
        - Use the model to impute the missing values
    - At each iteration we replace the mode-imputed values with RF-imputed values.
- The IVEware implementation is available in several platforms, but Python is not one of them.  I run it in R outside this notebook.  Be aware that the random selection of values to erase is different for each run, so the IVEware imputation must be run anew.  

# Results of Comparison of Three Imputation Methods

- We ran the imputation on 78 features with 224,850 samples.  
    - The features are the features of the CRSS dataset that are have data for all of 2016 - 2020, are not the results of imputation by CRSS, may have a pattern (not random numbers like VIN numbers), and that do not have more than 20% of the samples missing.  
    - The features were discretized (binned) down to 2-10 categories before imputation.
    - The samples are those of the 619,027 that have no missing values in any of the 78 features.
- First Run
    - Percentage of Samples Incorrectly Imputed

| | Percentage of Samples Incorrectly Imputed |
| --- | --- |
| Random Forest | 22.25% |
| Mode Imputation | 28.51% |
| IVEware | 24.23% |

    - Comparison of number of errors in the 78 features:

|  | Fewer | Equal | More | Total |
| --- | --- | --- | --- | --- |
Compare RF to Mode |  45 | 33 | 0 | 78 |
Compare RF to IVEware | 50 | 0 | 28 | 78 |
Compare Mode to IVEware | 39 | 0 | 39 |  78 |


- Second Run
    - Percentage of Samples Incorrectly Imputed

| | Percentage of Samples Incorrectly Imputed |
| --- | --- |
| Random Forest | 22.17 % |
| Mode Imputation | 28.42% |
| IVEware |  23.84% |


    - Comparison of number of errors in the 78 features:

|  | Fewer | Equal | More |
| --- | --- | --- | --- |
| Compare RF to Mode | 46 | 31 | 1 |
| Compare RF to IVEware | 49 | 0 | 29 |
| Compare Mode to IVEware |  36 | 1 | 41 |

    - Number of NaN Imputed Differently by Different Methods

|  |  |
| --- | --- |
|Total Number of NaN|  2,443,202|
|RF Different from Mode|  273,351|
|RF Different from IVEware|  606,751|
|Mode Different from IVEware|  738,833|



## Discussion

- Random Forest is as good or better than Mode for (nearly) every feature.
- Random Forest is as good or better than IVEware on more than half of the features, but not overwhelmingly, and slightly better in the count of missing samples correctly imputed.
- IVEware and Mode are comparable in the number of features, but IVEware is much better in the count of missing samples correctly imputed.
- Random Forest and Mode make the same mistakes.  
- IVEware makes different mistakes from Random Forest and Mode.

## Conclusion

- Use Random Forest

In [15]:
%%latex
\tableofcontents

<IPython.core.display.Latex object>

# Setup
## Import Libraries

In [2]:
import sys, copy, math, time, os

print ('Python version: {}'.format(sys.version))

import numpy as np
print ('NumPy version: {}'.format(np.__version__))
np.set_printoptions(suppress=True)


import pandas as pd
print ('Pandas version:  {}'.format(pd.__version__))
pd.set_option('display.max_rows', 500)

!pip install scikit-learn

import sklearn
print ('SciKit-Learn version: {}'.format(sklearn.__version__))
from sklearn.model_selection import train_test_split

import sklearn.neighbors._base
sys.modules['sklearn.neighbors.base'] = sklearn.neighbors._base

from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import RandomForestRegressor

# Set Randomness.  Copied from https://www.kaggle.com/code/abazdyrev/keras-nn-focal-loss-experiments
import random
#np.random.seed(42) # NumPy
#random.seed(42) # Python
#tf.set_random_seed(42) # Tensorflow

from IPython.display import Audio
sound_file = './beep.wav'

import warnings
warnings.filterwarnings('ignore')


Python version: 3.10.8 | packaged by conda-forge | (main, Nov 22 2022, 08:25:13) [Clang 14.0.6 ]
NumPy version: 1.24.0
Pandas version:  1.5.2
DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
SciKit-Learn version: 1.2.0


# Import Data

## Get Data
- The Get_Data_from_Original() reads the (original) CRSS files from the CRSS directory, preprocesses it, and writes it to files in a folder outside this GitHub repo (because the files are too large for my subscription), and returns the dataframes.
- The Get_Data_from_Temp_Files() reads the temp files and returns the dataframes.  I created this option for running repeatedly during writing and debugging, because it's much faster.

In [3]:
def Get_Data():
    print ('Get_Data')
    data = pd.read_csv('../../Big_Files/CRSS_Discretized_All_12_22_22.csv', low_memory=False)
    print ('data.shape = ', data.shape)
    print ('Drop Imputed Columns')
    for feature in data:
        if '_IM' in feature:
#            print (feature)
            data.drop(columns=feature, inplace=True)
    
    print ('data.shape = ', data.shape)
    print ()
    
    return data

In [4]:
#data = Get_Data()


In [5]:
def Impute_Round_Robin(data):
    print ('Impute()')
    pd.set_option('display.max_columns', None)
    
    # Replace 'Unknown' with np.NaN
    data.replace({'Unknown': np.nan}, inplace=True)
    display(data.head(20))
    print ()
    
#    data.sort_values(by = ['CASENUM', 'VEH_NO', 'PER_NO'], ascending = [True, True, True])
    
    # Make a list of features with missing samples, 
    #     ordered by the number of missing samples, 
    #     from least to most.  
    Missing = []
    Complete = []
    for feature in data:
        s = data[feature].isna().sum()
        if s==0:
            Complete.append([feature, s])
        if s>0:
            Missing.append([feature, s])
    Missing = sorted (Missing, key=lambda x:x[1], reverse=False)
    print ()
    print ('Complete[]')
    display(Complete)
    print ()
    print ('Missing[]')
    display(Missing)
    print ()
    
    print ('Make data_Mode')
    print ()
    data_Mode = pd.DataFrame()
    for X in Complete:
        feature = X[0]
        data_Mode[feature] = data[feature]
    for M in Missing:
        feature = M[0]
        m = data[feature].mode()[0]
        print (feature, M[1], m)
        data_Mode[feature] = data[feature].fillna(m)
    print ('data_Mode')
    display(data_Mode.head(20))
#    data.sort_values(
#        by = ['CASENUM', 'VEH_NO', 'PER_NO'], 
#        ascending = [True, True, True], 
#        inplace=True
#    )
#    print ()
#    print ('data.PER_NO.equals(data__Mode.PER_NO)')
#    print (data.PER_NO.equals(data_Mode.PER_NO))
#    print ()
#    
    print ()
    print ('Make starting point for data_Imputed')
    data_Imputed = pd.DataFrame()
    for X in Complete:
        feature = X[0]
        data_Imputed[feature] = data[feature]
    for X in Missing:
        feature = X[0]
        data_Imputed[feature] = data_Mode[feature]
    print ('data_Imputed')
    display(data_Imputed.head(20))
    print ()
#    data_Imputed.sort_values(
#        by = ['CASENUM', 'VEH_NO', 'PER_NO'], 
#        ascending = [True, True, True], 
#        inplace=True
#    )
#    print ()
#    print ('data.PER_NO.equals(data_Imputed.PER_NO)')
#    print (data.PER_NO.equals(data_Imputed.PER_NO))
#    print ()
    
    print ('Start Loop')
    print ()
    n = 0
    for M in Missing:
        n += 1
        print (M)
        feature = M[0]
        data_Imputed[feature] = data[feature]
#        print ()
#        print ('data[feature].isna().sum()')
#        print (data[feature].isna().sum())
#        print ('data_Imputed[feature].isna().sum()')
#        print (data_Imputed[feature].isna().sum())
#        print ()
        W = data_Imputed.dropna(subset=[feature])
        X = data_Imputed.dropna(subset=[feature])
        y = X[feature]
        X.drop(columns=feature, inplace=True)
        Z = data_Imputed[data_Imputed[feature].isna()]
        Z.drop(columns=feature, inplace=True)
#        Z.reset_index(drop=True, inplace=True)
#        print (data.shape)
#        print (X.shape)
#        display(X.head(40))
#        display(y.head(40))
#        print (Z.shape)
#        display(Z)
        clf = RandomForestClassifier(max_depth=2, random_state=0)
        clf.fit(X,y)
#        print ('clf.predict(Z)')
        z = clf.predict(Z)
        print (len(z))
        display(z)
        Z[feature] = z
#        display(Z)
        data_Imputed = pd.concat([Z, W])
#        display(data_Imputed.head(60))
        print (data_Imputed.shape)
        print ()
#        data_Imputed.sort_values(
#            by = ['CASENUM', 'VEH_NO', 'PER_NO'], 
#            ascending = [True, True, True], 
#            inplace=True
#        )
#        print ()
#        print ('data.PER_NO.equals(data_Imputed.PER_NO)')
#        print (data.PER_NO.equals(data_Imputed.PER_NO))
#        print ()
               
        Check_Feature(data, data_Imputed, feature)
#        if n==10:
#            return data_Imputed
    
    
    
    
    print ()
    return data_Imputed

In [6]:
def Impute_Full(data):
    print ('Impute()')
    data.replace({'Unknown': np.nan}, inplace=True)
    for feature in data:
        print (feature, len(pd.unique(data[feature])))
    print ()
    mf = MissForest()
    data = mf.fit_transform(data)
    return data

In [7]:
def Check(data, data_Imputed):
    Features = data.columns
    print (Features)
    for feature in Features:
        U = pd.unique(data[feature]).tolist()
        print (U)
        A = []
        for u in U:
            a = len(data[data[feature]==u])
            b = len(data_Imputed[data_Imputed[feature]==u])
            A.append([u, a, b])
        display(A)
        print ()


In [8]:
def Check_Feature(data, data_Imputed, feature):
    U = pd.unique(data[feature]).tolist()
    U = [x for x in U if x == x]
    print (U)
    A = []
    for u in U:
        a = len(data[data[feature]==u])
        b = len(data_Imputed[data_Imputed[feature]==u])
        A.append([u, a, b, b-a])
    a = data[feature].isna().sum()
    b = data_Imputed[feature].isna().sum()
    A.append(['NaN', a, b, 0])
    A = pd.DataFrame(A, columns=['Value', 'Original', 'Imputed', 'Difference'])
    display(A)
    print ()


# Test_Accuracy

In [13]:
def Compare_Imputation_Methods_Part_1():
    print ()
    print ('Compare_Imputation_Methods_Part_1()')
    data = Get_Data()
    data.drop(columns=['CASENUM', 'VEH_NO', 'PER_NO'], inplace=True)
    print (data.shape)

    # Drop all samples with missing data, so we have ground truth
    data.replace({'Unknown':np.nan}, inplace=True)
    data.dropna(inplace=True)
    data.reset_index(inplace=True, drop=True)
    for feature in data:
        data[feature] = pd.to_numeric(data[feature])
    data.astype('int64')

    data_Ground_Truth = data.copy(deep=True)
    for feature in data_Ground_Truth:
        data_Ground_Truth[feature] = pd.to_numeric(data_Ground_Truth[feature])
    data_Ground_Truth = data_Ground_Truth.astype('int64')
    print ('data_Ground_Truth.shape')
    print (data_Ground_Truth.shape)
    display(data_Ground_Truth.head())

    # Randomly pick 15% of the values from each row
    # and set them to be missing
    print ('Remove 15% of values from each row')
    frac = .15
    N = data.shape[0] * frac # Number of NaN in each feature
    for c in data.columns:
        idx = np.random.choice(a=data.index, size=int(len(data) * frac))
        data.loc[idx, c] = np.nan
    data_NaN = data.copy(deep=True)
    print ('data_NaN.shape')
    print (data_NaN.shape)
    display(data_NaN.head())

    data_IVEware = data.fillna('')
    data_IVEware.to_csv('../../Big_Files/data_IVEware.txt', sep='\t', index=False)
    
    data_Mode = pd.DataFrame()
    for feature in data:
        data_Mode[feature] = data[feature].fillna(data[feature].mode()[0])
    data_Mode = data_Mode.astype('int64')
    print ('data_Mode.shape')
    print (data_Mode.shape)
    display(data_Mode.head())
    
    data_RF = Impute_Round_Robin(data)
    data_RF.sort_index(inplace=True)
    data_RF = data_RF[data.columns]  
    data_RF = data_RF.astype('int64')
    
    print ('data_RF.shape')
    print (data_RF.shape)
    display(data_RF.head())
#    print ()

    return data_Ground_Truth, data_NaN, data_RF, data_Mode

def Compare_Imputation_Methods_Part_2(
    data_Ground_Truth, data_NaN, data_RF, data_Mode, data_IVEware
):
    print ('Compare_Imputation_Methods_Part_2')
    A = []
    for feature in data_NaN:
        N = data_NaN[feature].isna().sum()
#        print (feature, N)
#        print ()
        D = data_Ground_Truth[feature] != data_RF[feature]
        d = D.sum()
        E = data_Ground_Truth[feature] != data_Mode[feature]
        e = E.sum()
        F = data_Ground_Truth[feature] != data_IVEware[feature]
        f = F.sum()
        G = data_RF[feature] != data_Mode[feature]
        g = G.sum()
        H = data_RF[feature] != data_IVEware[feature]
        h = H.sum()
        I = data_Mode[feature] != data_IVEware[feature]
        i = I.sum()
        print (feature, N, d, e, f, g, h, i)
        print (
            feature, 
            data_Ground_Truth.dtypes[feature],
            data_NaN.dtypes[feature],
            data_RF.dtypes[feature],
            data_Mode.dtypes[feature],
            data_IVEware.dtypes[feature],
        )
        A.append([
            feature, N, 
            d, int(d/N*100), 
            e, int(e/N*100), 
            f, int(f/N*100),
            g, int(g/N*100),
            h, int(h/N*100),
            i, int(i/N*100),
        ])
#        print (D[:10])
        print ()
    print ()
    
    A = sorted(A, key=lambda x:x[3])
    B = pd.DataFrame(
        A, 
        columns=[
            'Feature', 'nNaN', 
            'nRF Incorrect', 'pRF Incorrect', 
            'nMode Incorrect', 'pMode Incorrect', 
            'nIVEware Incorrect', 'pIVEware Incorrect',
            'RF and Mode Different', 'RF v/s Mode %',
            'RF and IVEware Different', 'RF v/s IVEware %',
            'Mode and IVEware Different', 'Mode v/s IVEware %'
        ]
    )
    display(B)
    a = sum([x[1] for x in A])
    b = sum([x[2] for x in A])
    c = sum([x[4] for x in A])
    d = sum([x[6] for x in A])
    e = round(b/a*100,2)
    f = round(c/a*100,2)
    g = round(d/a*100,2)
    s = len(A) - sum([x[8] for x in A])
    t = len(A) - sum([x[9] for x in A])
    u = len(A) - sum([x[10] for x in A])

    RF_less_Mode = sum([x[2] < x[4] for x in A])
    RF_equal_Mode = sum([x[2] == x[4] for x in A])
    RF_greater_Mode = sum([x[2] > x[4] for x in A])

    RF_less_IVEware = sum([x[2] < x[6] for x in A])
    RF_equal_IVEware = sum([x[2] == x[6] for x in A])
    RF_greater_IVEware = sum([x[2] > x[6] for x in A])

    Mode_less_IVEware = sum([x[4] < x[6] for x in A])
    Mode_equal_IVEware = sum([x[4] == x[6] for x in A])
    Mode_greater_IVEware = sum([x[4] > x[6] for x in A])

    print ()
    print ('Error RF = ', e)
    print ('Error Mode = ', f)
    print ('Error IVEware = ', g)
    print ('nRF > nMode: ', s)
    print ('nRF > nIVEware: ', t)
    print ('nModel > nIVEware: ', u)
    print ('Compare RF to Mode: ', RF_less_Mode, RF_equal_Mode, RF_greater_Mode)
    print ('Compare RF to IVEware: ', RF_less_IVEware, RF_equal_IVEware, RF_greater_IVEware)
    print ('Compare Mode to IVEware: ', Mode_less_IVEware, Mode_equal_IVEware, Mode_greater_IVEware)
    print ()
    print ('Number of NaN in data_NaN: ', data_NaN.isna().sum().sum())
    print ('RF Different from Mode: ', sum([x[8] for x in A]))
    print ('RF Different from IVEware: ', sum([x[10] for x in A]))
    print ('Mode Different from IVEware: ', sum([x[12] for x in A]))
        
    display(Audio(sound_file, autoplay=True))
    
    
        

In [10]:
data_Ground_Truth, data_NaN, data_RF, data_Mode = Compare_Imputation_Methods_Part_1()


Compare_Imputation_Methods_Part_1()
Get_Data
data.shape =  (619027, 105)
Drop Imputed Columns
data.shape =  (619027, 81)

(619027, 78)
data_Ground_Truth.shape
(224850, 78)


,HOUR,INT_HWY,LGT_COND,MONTH,PEDS,PERMVIT,REL_ROAD,RELJCT2,SCH_BUS,URBANICITY,...,ALC_STATUS,EJECTION,HOSPITAL,INJ_SEV,PER_TYP,REST_MIS,REST_USE,SEAT_POS,SEX,VEH_AGE
0,3,0,3,0,0,1,1,1,0,2,...,1,0,8,0,2,1,0,3,1,0
1,1,0,3,0,0,2,1,0,0,2,...,1,1,0,3,2,1,1,3,1,4
2,3,0,2,0,0,2,0,1,0,2,...,1,1,0,3,2,1,1,3,0,3
3,3,0,2,0,0,2,0,1,0,2,...,1,1,5,0,1,1,1,1,0,3
4,3,0,2,0,0,2,0,1,0,2,...,1,1,0,3,1,1,1,4,1,3


Remove 15% of values from each row
data_NaN.shape
(224850, 78)


,HOUR,INT_HWY,LGT_COND,MONTH,PEDS,PERMVIT,REL_ROAD,RELJCT2,SCH_BUS,URBANICITY,...,ALC_STATUS,EJECTION,HOSPITAL,INJ_SEV,PER_TYP,REST_MIS,REST_USE,SEAT_POS,SEX,VEH_AGE
0,3.0,0.0,3.0,0.0,0.0,NaN,1.0,1.0,0.0,2.0,...,NaN,0.0,NaN,NaN,2.0,1.0,0.0,3.0,1.0,0.0
1,1.0,0.0,3.0,0.0,0.0,2.0,1.0,0.0,0.0,2.0,...,1.0,1.0,0.0,3.0,2.0,1.0,1.0,3.0,1.0,4.0
2,3.0,0.0,2.0,NaN,0.0,2.0,0.0,1.0,0.0,2.0,...,1.0,1.0,0.0,3.0,2.0,1.0,1.0,3.0,0.0,3.0
3,3.0,0.0,2.0,0.0,0.0,2.0,0.0,NaN,0.0,2.0,...,1.0,1.0,5.0,0.0,1.0,1.0,1.0,NaN,0.0,NaN
4,3.0,0.0,2.0,0.0,0.0,2.0,0.0,NaN,0.0,2.0,...,1.0,NaN,NaN,3.0,1.0,1.0,1.0,4.0,1.0,3.0


data_Mode.shape
(224850, 78)


...

array([0., 0., 0., ..., 0., 0., 0.])

(224850, 78)

[0.0, 1.0, 2.0]


,Value,Original,Imputed,Difference
0,0.0,188349,219778,31429
1,1.0,4899,4899,0
2,2.0,173,173,0
3,NaN,31429,0,0




data_RF.shape
(224850, 78)


,HOUR,INT_HWY,LGT_COND,MONTH,PEDS,PERMVIT,REL_ROAD,RELJCT2,SCH_BUS,URBANICITY,VE_TOTAL,WEATHER,DAY_WEEK,WRK_ZONE,VE_FORMS,PVH_INVL,PERNOTMVIT,NUM_INJ,PSU,PJ,MAN_COLL,HARM_EV,TYP_INT,YEAR,REGION,ALCOHOL,MAX_SEV,RELJCT1,ACC_TYPE,BODY_TYP,BUS_USE,CARGO_BT,DR_PRES,EMER_USE,FIRE_EXP,HAZ_CNO,HAZ_INV,HAZ_PLAC,HAZ_REL,HIT_RUN,IMPACT1,J_KNIFE,M_HARM,MAK_MOD,MAKE,MAX_VSEV,MODEL,NUM_INJV,NUMOCCS,P_CRASH1,P_CRASH2,PCRASH4,PCRASH5,ROLLOVER,SPEC_USE,SPEEDREL,TOW_VEH,TOWED,VALIGN,VEH_ALCH,VPROFILE,VSPD_LIM,VSURCOND,VTCONT_F,VTRAFCON,VTRAFWAY,AIR_BAG,ALC_RES,ALC_STATUS,EJECTION,HOSPITAL,INJ_SEV,PER_TYP,REST_MIS,REST_USE,SEAT_POS,SEX,VEH_AGE
0,3,0,3,0,0,2,1,1,0,2,2,1,1,0,2,0,0,1,0,2,4,1,1,2016,4,2,3,0,4,0,1,0,1,1,1,1,0,0,1,0,5,1,1,0,0,0,0,1,3,1,3,1,2,1,1,1,0,0,1,1,1,0,1,1,1,2,0,0,1,0,0,3,2,1,0,3,1,0
1,1,0,3,0,0,2,1,0,0,2,2,1,1,0,2,0,0,2,1,3,2,1,0,2016,3,2,1,0,2,5,1,0,1,1,1,1,0,0,1,0,1,1,1,4,6,2,4,3,3,1,1,1,3,1,1,1,0,0,1,1,1,1,1,1,1,0,0,0,1,1,0,3,2,1,1,3,1,4
2,3,0,2,0,0,2,0,1,0,2,1,2,1,0,1,0,0,1,1,1,1,3,1,2016,3,2,2,0,0,3,1,0,1,1,1,1,0,0,1,0,1,1,0,0,8,0,2,1,5,1,0,1,1,1,1,0,0,0,1,1,1,5,2,1,1,3,0,0,1,1,0,3,2,1,1,3,0,3
3,3,0,2,0,0,2,0,1,0,2,1,2,1,0,1,0,0,1,1,1,1,3,1,2016,3,2,2,0,0,3,1,0,1,1,1,1,0,0,1,0,1,1,0,0,8,0,1,1,5,1,0,0,1,1,1,0,0,0,1,1,1,5,2,1,1,3,0,0,1,1,5,0,1,1,1,3,0,2
4,3,0,2,0,0,2,0,1,0,2,1,2,1,0,1,0,0,1,1,1,1,3,1,2016,3,2,2,0,0,3,1,0,1,1,1,1,0,0,1,0,1,1,0,0,0,0,1,1,5,1,0,0,1,1,1,0,0,0,1,1,1,5,2,1,1,3,0,0,1,1,0,3,1,1,1,4,1,3


## Now do IVEware Imputation

In [11]:
data_IVEware = pd.read_csv('../../Big_Files/data_IVEware.csv')
data_IVEware.drop(columns='Unnamed: 0', inplace=True)

print ('data_Ground_Truth', data_Ground_Truth.shape)
display(data_Ground_Truth.head(10))
print ('data_NaN', data_NaN.shape)
display(data_NaN.head(10))
print ('data_RF', data_RF.shape)
display(data_RF.head(10))
print ('data_Mode', data_Mode.shape)
display(data_Mode.head(10))
print ('data_IVEware', data_IVEware.shape)
display(data_IVEware.head(10))


data_Ground_Truth (224850, 78)


,HOUR,INT_HWY,LGT_COND,MONTH,PEDS,PERMVIT,REL_ROAD,RELJCT2,SCH_BUS,URBANICITY,VE_TOTAL,WEATHER,DAY_WEEK,WRK_ZONE,VE_FORMS,PVH_INVL,PERNOTMVIT,NUM_INJ,PSU,PJ,MAN_COLL,HARM_EV,TYP_INT,YEAR,REGION,ALCOHOL,MAX_SEV,RELJCT1,ACC_TYPE,BODY_TYP,BUS_USE,CARGO_BT,DR_PRES,EMER_USE,FIRE_EXP,HAZ_CNO,HAZ_INV,HAZ_PLAC,HAZ_REL,HIT_RUN,IMPACT1,J_KNIFE,M_HARM,MAK_MOD,MAKE,MAX_VSEV,MODEL,NUM_INJV,NUMOCCS,P_CRASH1,P_CRASH2,PCRASH4,PCRASH5,ROLLOVER,SPEC_USE,SPEEDREL,TOW_VEH,TOWED,VALIGN,VEH_ALCH,VPROFILE,VSPD_LIM,VSURCOND,VTCONT_F,VTRAFCON,VTRAFWAY,AIR_BAG,ALC_RES,ALC_STATUS,EJECTION,HOSPITAL,INJ_SEV,PER_TYP,REST_MIS,REST_USE,SEAT_POS,SEX,VEH_AGE
0,3,0,3,0,0,1,1,1,0,2,2,1,1,0,2,0,0,1,0,2,4,1,1,2016,4,2,3,0,4,0,1,0,1,1,1,1,0,0,1,0,5,1,1,0,0,0,0,1,3,5,2,1,2,1,1,1,0,0,1,1,1,0,1,1,1,2,0,0,1,0,8,0,2,1,0,3,1,0
1,1,0,3,0,0,2,1,0,0,2,2,1,1,0,2,0,0,2,1,3,2,1,0,2016,3,2,1,0,1,5,1,0,1,1,1,1,0,0,1,0,1,1,1,4,6,2,4,3,3,1,1,1,3,1,1,1,0,0,1,1,1,1,1,1,1,0,0,0,1,1,0,3,2,1,1,3,1,4
2,3,0,2,0,0,2,0,1,0,2,1,2,1,0,1,0,0,1,1,1,1,3,1,2016,3,2,2,0,0,3,1,0,1,1,1,1,0,0,1,0,1,1,0,0,8,0,2,1,5,1,0,0,1,1,1,0,0,0,1,1,1,5,2,1,1,3,0,0,1,1,0,3,2,1,1,3,0,3
3,3,0,2,0,0,2,0,1,0,2,1,2,1,0,1,0,0,1,1,1,1,3,1,2016,3,2,2,0,0,3,1,0,1,1,1,1,0,0,1,0,1,1,0,0,8,0,2,1,5,1,0,0,1,1,1,0,0,0,1,1,1,5,2,1,1,3,0,0,1,1,5,0,1,1,1,1,0,3
4,3,0,2,0,0,2,0,1,0,2,1,2,1,0,1,0,0,1,1,1,1,3,1,2016,3,2,2,0,0,3,1,0,1,1,1,1,0,0,1,0,1,1,0,0,8,0,2,1,5,1,0,0,1,1,1,0,0,0,1,1,1,5,2,1,1,3,0,0,1,1,0,3,1,1,1,4,1,3
5,3,0,3,0,0,2,1,1,1,1,2,1,1,0,2,0,0,1,4,2,3,1,1,2016,2,2,2,0,4,5,1,0,1,1,1,1,0,0,1,0,1,1,1,4,6,2,4,3,3,1,5,1,3,1,1,1,0,2,1,1,1,4,2,1,1,0,1,0,1,1,0,3,2,1,2,3,1,1
6,3,0,3,0,0,2,1,1,1,1,2,1,1,0,2,0,0,1,4,2,3,1,1,2016,2,2,2,0,3,5,2,1,1,1,1,1,0,0,1,0,4,1,1,4,8,0,4,1,1,4,3,1,3,1,2,1,0,2,1,1,1,4,2,1,1,0,1,0,1,1,0,3,2,1,2,3,1,2
7,3,0,3,0,0,2,1,1,1,1,2,1,1,0,2,0,0,1,4,2,3,1,1,2016,2,2,2,0,3,5,2,1,1,1,1,1,0,0,1,0,4,1,1,4,8,0,4,1,1,4,3,1,3,1,2,1,0,2,1,1,1,4,2,1,1,0,0,0,1,1,0,0,1,1,0,0,1,2
8,2,0,3,0,0,1,1,1,0,2,2,3,1,0,2,0,0,0,1,3,3,1,1,2016,3,9,0,0,3,1,1,0,1,1,1,1,0,0,1,0,4,1,1,2,0,2,1,3,3,4,3,1,3,1,1,1,0,2,1,1,1,5,1,1,1,2,1,0,1,1,0,3,2,1,1,3,0,0
9,2,0,3,0,0,0,0,1,0,2,1,3,1,0,1,0,0,0,1,3,1,3,1,2016,3,2,0,0,0,5,1,0,1,1,1,1,0,0,1,0,0,1,0,3,8,2,4,3,3,1,0,0,1,1,1,0,0,0,1,1,2,7,1,1,1,0,1,0,1,1,0,3,2,1,1,3,1,2


data_NaN (224850, 78)


,HOUR,INT_HWY,LGT_COND,MONTH,PEDS,PERMVIT,REL_ROAD,RELJCT2,SCH_BUS,URBANICITY,VE_TOTAL,WEATHER,DAY_WEEK,WRK_ZONE,VE_FORMS,PVH_INVL,PERNOTMVIT,NUM_INJ,PSU,PJ,MAN_COLL,HARM_EV,TYP_INT,YEAR,REGION,ALCOHOL,MAX_SEV,RELJCT1,ACC_TYPE,BODY_TYP,BUS_USE,CARGO_BT,DR_PRES,EMER_USE,FIRE_EXP,HAZ_CNO,HAZ_INV,HAZ_PLAC,HAZ_REL,HIT_RUN,IMPACT1,J_KNIFE,M_HARM,MAK_MOD,MAKE,MAX_VSEV,MODEL,NUM_INJV,NUMOCCS,P_CRASH1,P_CRASH2,PCRASH4,PCRASH5,ROLLOVER,SPEC_USE,SPEEDREL,TOW_VEH,TOWED,VALIGN,VEH_ALCH,VPROFILE,VSPD_LIM,VSURCOND,VTCONT_F,VTRAFCON,VTRAFWAY,AIR_BAG,ALC_RES,ALC_STATUS,EJECTION,HOSPITAL,INJ_SEV,PER_TYP,REST_MIS,REST_USE,SEAT_POS,SEX,VEH_AGE
0,3.0,0.0,3.0,0.0,0.0,NaN,1.0,1.0,0.0,2.0,2.0,1.0,1.0,0.0,2.0,0.0,NaN,1.0,0.0,2.0,4.0,1.0,1.0,2016.0,4.0,2.0,3.0,0.0,4.0,0.0,1.0,0.0,1.0,1.0,1.0,1.0,NaN,0.0,NaN,0.0,5.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,3.0,NaN,NaN,1.0,2.0,1.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,2.0,0.0,0.0,NaN,0.0,NaN,NaN,2.0,1.0,0.0,3.0,1.0,0.0
1,1.0,0.0,3.0,0.0,0.0,2.0,1.0,0.0,0.0,2.0,2.0,1.0,1.0,NaN,NaN,0.0,0.0,2.0,1.0,3.0,2.0,1.0,0.0,2016.0,3.0,NaN,1.0,0.0,NaN,5.0,1.0,0.0,1.0,1.0,1.0,1.0,NaN,0.0,1.0,0.0,NaN,1.0,1.0,4.0,6.0,2.0,4.0,3.0,3.0,1.0,NaN,1.0,3.0,1.0,1.0,NaN,NaN,NaN,1.0,NaN,1.0,1.0,NaN,1.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,3.0,2.0,1.0,1.0,3.0,1.0,4.0
2,3.0,0.0,2.0,NaN,0.0,2.0,0.0,1.0,0.0,2.0,1.0,2.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0,3.0,1.0,2016.0,3.0,2.0,2.0,0.0,0.0,3.0,1.0,0.0,NaN,1.0,1.0,1.0,0.0,0.0,1.0,0.0,1.0,NaN,0.0,0.0,8.0,0.0,2.0,1.0,5.0,1.0,0.0,NaN,1.0,1.0,1.0,0.0,0.0,0.0,1.0,1.0,1.0,5.0,2.0,1.0,NaN,3.0,0.0,0.0,1.0,1.0,0.0,3.0,2.0,1.0,1.0,3.0,0.0,3.0
3,3.0,0.0,2.0,0.0,0.0,2.0,0.0,NaN,0.0,2.0,1.0,2.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0,3.0,1.0,2016.0,3.0,2.0,2.0,0.0,0.0,3.0,1.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,8.0,0.0,NaN,1.0,5.0,1.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,NaN,1.0,1.0,5.0,2.0,1.0,NaN,3.0,0.0,0.0,1.0,1.0,5.0,0.0,1.0,1.0,1.0,NaN,0.0,NaN
4,3.0,0.0,2.0,0.0,0.0,2.0,0.0,NaN,0.0,2.0,1.0,2.0,1.0,0.0,1.0,0.0,NaN,1.0,1.0,1.0,1.0,3.0,1.0,2016.0,3.0,2.0,2.0,0.0,0.0,3.0,1.0,0.0,1.0,1.0,1.0,1.0,NaN,0.0,1.0,0.0,NaN,1.0,NaN,0.0,NaN,0.0,NaN,1.0,5.0,1.0,0.0,0.0,1.0,1.0,NaN,0.0,0.0,0.0,1.0,1.0,1.0,5.0,2.0,1.0,1.0,3.0,0.0,NaN,1.0,NaN,NaN,3.0,1.0,1.0,1.0,4.0,1.0,3.0
5,3.0,0.0,3.0,0.0,0.0,NaN,1.0,1.0,NaN,1.0,2.0,1.0,1.0,0.0,2.0,0.0,0.0,NaN,4.0,2.0,3.0,1.0,1.0,2016.0,2.0,2.0,2.0,0.0,4.0,5.0,1.0,NaN,1.0,1.0,NaN,NaN,0.0,0.0,1.0,0.0,1.0,1.0,1.0,4.0,6.0,2.0,4.0,3.0,3.0,1.0,5.0,1.0,3.0,1.0,1.0,1.0,0.0,2.0,1.0,1.0,1.0,4.0,2.0,NaN,1.0,0.0,NaN,0.0,1.0,1.0,0.0,3.0,2.0,1.0,2.0,3.0,1.0,1.0
6,3.0,0.0,3.0,0.0,0.0,NaN,1.0,1.0,1.0,1.0,2.0,1.0,1.0,0.0,2.0,0.0,0.0,1.0,NaN,2.0,3.0,1.0,1.0,2016.0,2.0,NaN,2.0,NaN,3.0,5.0,2.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,1.0,NaN,4.0,1.0,1.0,4.0,8.0,0.0,4.0,1.0,1.0,4.0,3.0,1.0,3.0,1.0,2.0,1.0,NaN,2.0,1.0,1.0,1.0,NaN,2.0,1.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,3.0,2.0,1.0,2.0,3.0,1.0,2.0
7,3.0,0.0,3.0,0.0,NaN,2.0,1.0,1.0,1.0,1.0,2.0,1.0,1.0,0.0,2.0,0.0,0.0,1.0,4.0,2.0,3.0,NaN,1.0,2016.0,2.0,2.0,2.0,0.0,3.0,5.0,2.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,1.0,NaN,4.0,1.0,1.0,NaN,8.0,0.0,4.0,NaN,1.0,4.0,3.0,1.0,3.0,1.0,2.0,1.0,0.0,2.0,1.0,1.0,1.0,4.0,2.0,NaN,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,NaN,0.0,1.0,2.0
8,2.0,0.0,NaN,NaN,0.0,1.0,1.0,1.0,0.0,2.0,2.0,3.0,1.0,0.0,2.0,NaN,0.0,0.0,1.0,3.0,NaN,1.0,NaN,2016.0,3.0,9.0,0.0,0.0,3.0,1.0,1.0,NaN,1.0,1.0,1.0,1.0,0.0,0.0,1.0,NaN,4.0,1.0,1.0,2.0,0.0,NaN,1.0,3.0,3.0,4.0,3.0,NaN,3.0,1.0,1.0,1.0,0.0,2.0,NaN,1.0,1.0,NaN,1.0,1.0,1.0,2.0,1.0,0.0,1.0,1.0,0.0,3.0,2.0,NaN,1.0,3.0,0.0,0.0
9,2.0,NaN,3.0,0.0,0.0,0.0,0.0,1.0,0.0,2.0,NaN,3.0,1.0,NaN,1.0,0.0,0.0,0.0,1.0,3.0,NaN,3.0,1.0,2016.0,3.0,2.0,0.0,0.0,0.0,5.0,1.0,0.0,1.0,1.0,NaN,1.0,NaN,0.0,1.0,0.0,NaN,1.0,0.0,NaN,8.0,2.0,4.0,3.0,3.0,NaN,0.0,0.0,1.0,1.0,1.0,0.0,0.0,NaN,NaN,NaN,2.0,7.0,1.0,1.0,1.0,0.0,NaN,0.0,1.0,1.0,0.0,3.0,2.0,1.0,1.0,3.0,NaN,2.0


data_RF (224850, 78)


,HOUR,INT_HWY,LGT_COND,MONTH,PEDS,PERMVIT,REL_ROAD,RELJCT2,SCH_BUS,URBANICITY,VE_TOTAL,WEATHER,DAY_WEEK,WRK_ZONE,VE_FORMS,PVH_INVL,PERNOTMVIT,NUM_INJ,PSU,PJ,MAN_COLL,HARM_EV,TYP_INT,YEAR,REGION,ALCOHOL,MAX_SEV,RELJCT1,ACC_TYPE,BODY_TYP,BUS_USE,CARGO_BT,DR_PRES,EMER_USE,FIRE_EXP,HAZ_CNO,HAZ_INV,HAZ_PLAC,HAZ_REL,HIT_RUN,IMPACT1,J_KNIFE,M_HARM,MAK_MOD,MAKE,MAX_VSEV,MODEL,NUM_INJV,NUMOCCS,P_CRASH1,P_CRASH2,PCRASH4,PCRASH5,ROLLOVER,SPEC_USE,SPEEDREL,TOW_VEH,TOWED,VALIGN,VEH_ALCH,VPROFILE,VSPD_LIM,VSURCOND,VTCONT_F,VTRAFCON,VTRAFWAY,AIR_BAG,ALC_RES,ALC_STATUS,EJECTION,HOSPITAL,INJ_SEV,PER_TYP,REST_MIS,REST_USE,SEAT_POS,SEX,VEH_AGE
0,3,0,3,0,0,2,1,1,0,2,2,1,1,0,2,0,0,1,0,2,4,1,1,2016,4,2,3,0,4,0,1,0,1,1,1,1,0,0,1,0,5,1,1,0,0,0,0,1,3,1,3,1,2,1,1,1,0,0,1,1,1,0,1,1,1,2,0,0,1,0,0,3,2,1,0,3,1,0
1,1,0,3,0,0,2,1,0,0,2,2,1,1,0,2,0,0,2,1,3,2,1,0,2016,3,2,1,0,2,5,1,0,1,1,1,1,0,0,1,0,1,1,1,4,6,2,4,3,3,1,1,1,3,1,1,1,0,0,1,1,1,1,1,1,1,0,0,0,1,1,0,3,2,1,1,3,1,4
2,3,0,2,0,0,2,0,1,0,2,1,2,1,0,1,0,0,1,1,1,1,3,1,2016,3,2,2,0,0,3,1,0,1,1,1,1,0,0,1,0,1,1,0,0,8,0,2,1,5,1,0,1,1,1,1,0,0,0,1,1,1,5,2,1,1,3,0,0,1,1,0,3,2,1,1,3,0,3
3,3,0,2,0,0,2,0,1,0,2,1,2,1,0,1,0,0,1,1,1,1,3,1,2016,3,2,2,0,0,3,1,0,1,1,1,1,0,0,1,0,1,1,0,0,8,0,1,1,5,1,0,0,1,1,1,0,0,0,1,1,1,5,2,1,1,3,0,0,1,1,5,0,1,1,1,3,0,2
4,3,0,2,0,0,2,0,1,0,2,1,2,1,0,1,0,0,1,1,1,1,3,1,2016,3,2,2,0,0,3,1,0,1,1,1,1,0,0,1,0,1,1,0,0,0,0,1,1,5,1,0,0,1,1,1,0,0,0,1,1,1,5,2,1,1,3,0,0,1,1,0,3,1,1,1,4,1,3
5,3,0,3,0,0,2,1,1,0,1,2,1,1,0,2,0,0,0,4,2,3,1,1,2016,2,2,2,0,4,5,1,0,1,1,1,1,0,0,1,0,1,1,1,4,6,2,4,3,3,1,5,1,3,1,1,1,0,2,1,1,1,4,2,1,1,0,1,0,1,1,0,3,2,1,2,3,1,1
6,3,0,3,0,0,2,1,1,1,1,2,1,1,0,2,0,0,1,3,2,3,1,1,2016,2,2,2,0,3,5,2,1,1,1,1,1,0,0,1,0,4,1,1,4,8,0,4,1,1,4,3,1,3,1,2,1,0,2,1,1,1,2,2,1,1,0,1,0,1,1,0,3,2,1,2,3,1,2
7,3,0,3,0,0,2,1,1,1,1,2,1,1,0,2,0,0,1,4,2,3,1,1,2016,2,2,2,0,3,5,2,1,1,1,1,1,0,0,1,0,4,1,1,4,8,0,4,1,1,4,3,1,3,1,2,1,0,2,1,1,1,4,2,1,1,0,0,0,1,1,0,0,1,1,1,0,1,2
8,2,0,3,2,0,1,1,1,0,2,2,3,1,0,2,0,0,0,1,3,3,1,1,2016,3,9,0,0,3,1,1,0,1,1,1,1,0,0,1,0,4,1,1,2,0,2,1,3,3,4,3,1,3,1,1,1,0,2,1,1,1,2,1,1,1,2,1,0,1,1,0,3,2,1,1,3,0,0
9,2,0,3,0,0,0,0,1,0,2,1,3,1,0,1,0,0,0,1,3,1,3,1,2016,3,2,0,0,0,5,1,0,1,1,1,1,0,0,1,0,1,1,0,4,8,2,4,3,3,1,0,0,1,1,1,0,0,0,1,1,2,7,1,1,1,0,1,0,1,1,0,3,2,1,1,3,1,2


data_Mode (224850, 78)


,HOUR,INT_HWY,LGT_COND,MONTH,PEDS,PERMVIT,REL_ROAD,RELJCT2,SCH_BUS,URBANICITY,VE_TOTAL,WEATHER,DAY_WEEK,WRK_ZONE,VE_FORMS,PVH_INVL,PERNOTMVIT,NUM_INJ,PSU,PJ,MAN_COLL,HARM_EV,TYP_INT,YEAR,REGION,ALCOHOL,MAX_SEV,RELJCT1,ACC_TYPE,BODY_TYP,BUS_USE,CARGO_BT,DR_PRES,EMER_USE,FIRE_EXP,HAZ_CNO,HAZ_INV,HAZ_PLAC,HAZ_REL,HIT_RUN,IMPACT1,J_KNIFE,M_HARM,MAK_MOD,MAKE,MAX_VSEV,MODEL,NUM_INJV,NUMOCCS,P_CRASH1,P_CRASH2,PCRASH4,PCRASH5,ROLLOVER,SPEC_USE,SPEEDREL,TOW_VEH,TOWED,VALIGN,VEH_ALCH,VPROFILE,VSPD_LIM,VSURCOND,VTCONT_F,VTRAFCON,VTRAFWAY,AIR_BAG,ALC_RES,ALC_STATUS,EJECTION,HOSPITAL,INJ_SEV,PER_TYP,REST_MIS,REST_USE,SEAT_POS,SEX,VEH_AGE
0,3,0,3,0,0,2,1,1,0,2,2,1,1,0,2,0,0,1,0,2,4,1,1,2016,4,2,3,0,4,0,1,0,1,1,1,1,0,0,1,0,5,1,1,0,0,0,0,1,3,1,3,1,2,1,1,1,0,0,1,1,1,0,1,1,1,2,0,0,1,0,0,3,2,1,0,3,1,0
1,1,0,3,0,0,2,1,0,0,2,2,1,1,0,2,0,0,2,1,3,2,1,0,2016,3,2,1,0,3,5,1,0,1,1,1,1,0,0,1,0,1,1,1,4,6,2,4,3,3,1,3,1,3,1,1,1,0,2,1,1,1,1,1,1,1,0,0,0,1,1,0,3,2,1,1,3,1,4
2,3,0,2,2,0,2,0,1,0,2,1,2,1,0,1,0,0,1,1,1,1,3,1,2016,3,2,2,0,0,3,1,0,1,1,1,1,0,0,1,0,1,1,0,0,8,0,2,1,5,1,0,1,1,1,1,0,0,0,1,1,1,5,2,1,1,3,0,0,1,1,0,3,2,1,1,3,0,3
3,3,0,2,0,0,2,0,1,0,2,1,2,1,0,1,0,0,1,1,1,1,3,1,2016,3,2,2,0,0,3,1,0,1,1,1,1,0,0,1,0,1,1,0,0,8,0,1,1,5,1,0,0,1,1,1,0,0,0,1,1,1,5,2,1,1,3,0,0,1,1,5,0,1,1,1,3,0,0
4,3,0,2,0,0,2,0,1,0,2,1,2,1,0,1,0,0,1,1,1,1,3,1,2016,3,2,2,0,0,3,1,0,1,1,1,1,0,0,1,0,1,1,1,0,0,0,1,1,5,1,0,0,1,1,1,0,0,0,1,1,1,5,2,1,1,3,0,0,1,1,0,3,1,1,1,4,1,3
5,3,0,3,0,0,2,1,1,0,1,2,1,1,0,2,0,0,0,4,2,3,1,1,2016,2,2,2,0,4,5,1,0,1,1,1,1,0,0,1,0,1,1,1,4,6,2,4,3,3,1,5,1,3,1,1,1,0,2,1,1,1,4,2,1,1,0,1,0,1,1,0,3,2,1,2,3,1,1
6,3,0,3,0,0,2,1,1,1,1,2,1,1,0,2,0,0,1,3,2,3,1,1,2016,2,2,2,0,3,5,2,1,1,1,1,1,0,0,1,0,4,1,1,4,8,0,4,1,1,4,3,1,3,1,2,1,0,2,1,1,1,2,2,1,1,0,1,0,1,1,0,3,2,1,2,3,1,2
7,3,0,3,0,0,2,1,1,1,1,2,1,1,0,2,0,0,1,4,2,3,1,1,2016,2,2,2,0,3,5,2,1,1,1,1,1,0,0,1,0,4,1,1,3,8,0,4,3,1,4,3,1,3,1,2,1,0,2,1,1,1,4,2,1,1,0,0,0,1,1,0,0,1,1,1,0,1,2
8,2,0,3,2,0,1,1,1,0,2,2,3,1,0,2,0,0,0,1,3,3,1,1,2016,3,9,0,0,3,1,1,0,1,1,1,1,0,0,1,0,4,1,1,2,0,2,1,3,3,4,3,1,3,1,1,1,0,2,1,1,1,2,1,1,1,2,1,0,1,1,0,3,2,1,1,3,0,0
9,2,0,3,0,0,0,0,1,0,2,2,3,1,0,1,0,0,0,1,3,3,3,1,2016,3,2,0,0,0,5,1,0,1,1,1,1,0,0,1,0,1,1,0,3,8,2,4,3,3,1,0,0,1,1,1,0,0,2,1,1,2,7,1,1,1,0,1,0,1,1,0,3,2,1,1,3,1,2


data_IVEware (224850, 78)


,HOUR,INT_HWY,LGT_COND,MONTH,PEDS,PERMVIT,REL_ROAD,RELJCT2,SCH_BUS,URBANICITY,VE_TOTAL,WEATHER,DAY_WEEK,WRK_ZONE,VE_FORMS,PVH_INVL,PERNOTMVIT,NUM_INJ,PSU,PJ,MAN_COLL,HARM_EV,TYP_INT,YEAR,REGION,ALCOHOL,MAX_SEV,RELJCT1,ACC_TYPE,BODY_TYP,BUS_USE,CARGO_BT,DR_PRES,EMER_USE,FIRE_EXP,HAZ_CNO,HAZ_INV,HAZ_PLAC,HAZ_REL,HIT_RUN,IMPACT1,J_KNIFE,M_HARM,MAK_MOD,MAKE,MAX_VSEV,MODEL,NUM_INJV,NUMOCCS,P_CRASH1,P_CRASH2,PCRASH4,PCRASH5,ROLLOVER,SPEC_USE,SPEEDREL,TOW_VEH,TOWED,VALIGN,VEH_ALCH,VPROFILE,VSPD_LIM,VSURCOND,VTCONT_F,VTRAFCON,VTRAFWAY,AIR_BAG,ALC_RES,ALC_STATUS,EJECTION,HOSPITAL,INJ_SEV,PER_TYP,REST_MIS,REST_USE,SEAT_POS,SEX,VEH_AGE
0,3,0,3,0,0,2,1,1,0,2,2,1,1,0,2,0,0,1,0,2,4,1,1,2016,4,2,3,0,4,0,1,0,1,1,1,1,0,0,2,0,5,1,1,0,0,0,0,1,3,5,5,1,2,1,1,1,0,0,1,1,1,0,1,1,1,2,0,0,1,0,5,0,2,1,0,3,1,0
1,1,0,3,0,0,2,1,0,0,2,2,1,1,0,2,0,0,2,1,3,2,1,0,2016,3,2,1,0,4,5,1,0,1,1,1,1,0,0,1,0,1,1,1,4,6,2,4,3,3,1,5,1,3,1,1,1,0,0,1,1,1,1,1,1,1,0,0,0,1,1,0,3,2,1,1,3,1,4
2,3,0,2,1,0,2,0,1,0,2,1,2,1,0,1,0,0,1,1,1,1,3,1,2016,3,2,2,0,0,3,1,0,1,1,1,1,0,0,1,0,1,1,0,0,8,0,2,1,5,1,0,1,1,1,1,0,0,0,1,1,1,5,2,1,1,3,0,0,1,1,0,3,2,1,1,3,0,3
3,3,0,2,0,0,2,0,1,0,2,1,2,1,0,1,0,0,1,1,1,1,3,1,2016,3,2,2,0,0,3,1,0,1,1,1,1,0,0,1,0,1,1,0,0,8,0,2,1,5,1,0,0,1,1,1,0,0,0,1,1,1,5,2,1,1,3,0,0,1,1,5,0,1,1,1,1,0,4
4,3,0,2,0,0,2,0,1,0,2,1,2,1,0,1,0,0,1,1,1,1,3,1,2016,3,2,2,0,0,3,1,0,1,1,1,1,0,0,1,0,1,1,0,0,1,0,2,1,5,1,0,0,1,1,1,0,0,0,1,1,1,5,2,1,1,3,0,0,1,1,0,3,1,1,1,4,1,3
5,3,0,3,0,0,1,1,1,0,1,2,1,1,0,2,0,0,2,4,2,3,1,1,2016,2,2,2,0,4,5,1,0,1,1,1,1,0,0,1,0,1,1,1,4,6,2,4,3,3,1,5,1,3,1,1,1,0,2,1,1,1,4,2,2,1,0,1,0,1,1,0,3,2,1,2,3,1,1
6,3,0,3,0,0,2,1,1,1,1,2,1,1,0,2,0,0,1,3,2,3,1,1,2016,2,2,2,0,3,5,2,1,1,1,1,1,0,0,1,0,4,1,1,4,8,0,4,1,1,4,3,1,3,1,2,1,0,2,1,1,1,4,2,1,1,0,1,0,1,1,0,3,2,1,2,3,1,2
7,3,0,3,0,0,2,1,1,1,1,2,1,1,0,2,0,0,1,4,2,3,1,1,2016,2,2,2,0,3,5,2,1,1,1,1,1,0,0,1,0,4,1,1,4,8,0,4,1,1,4,3,1,3,1,2,1,0,2,1,1,1,4,2,2,1,0,0,0,1,1,0,0,1,1,2,0,1,2
8,2,0,3,2,0,1,1,1,0,2,2,3,1,0,2,0,0,0,1,3,3,1,1,2016,3,9,0,0,3,1,1,0,1,1,1,1,0,0,1,0,4,1,1,2,0,2,1,3,3,4,3,1,3,1,1,1,0,2,1,1,1,2,1,1,1,2,1,0,1,1,0,3,2,1,1,3,0,0
9,2,0,3,0,0,0,0,1,0,2,1,3,1,0,1,0,0,0,1,3,1,3,1,2016,3,2,0,0,0,5,1,0,1,1,1,1,0,0,1,0,4,1,0,4,8,2,4,3,3,1,0,0,1,1,1,0,0,2,1,1,2,7,1,1,1,0,1,0,1,1,0,3,2,1,1,3,1,2


In [14]:
Compare_Imputation_Methods_Part_2(
    data_Ground_Truth, data_NaN, data_RF, data_Mode, data_IVEware
)

Compare_Imputation_Methods_Part_2
HOUR 31324 22288 22492 23069 774 22556 22589
HOUR int64 float64 int64 int64 int64

INT_HWY 31335 3527 3527 3765 0 3602 3602
INT_HWY int64 float64 int64 int64 int64

LGT_COND 31292 8146 8146 8397 0 8132 8132
LGT_COND int64 float64 int64 int64 int64

MONTH 31255 18853 20482 20866 6631 20577 20442
MONTH int64 float64 int64 int64 int64

PEDS 31429 795 795 364 0 804 804
PEDS int64 float64 int64 int64 int64

PERMVIT 31342 10696 11936 8141 1558 10493 10914
PERMVIT int64 float64 int64 int64 int64

REL_ROAD 31306 919 2761 417 1859 895 2720
REL_ROAD int64 float64 int64 int64 int64

RELJCT2 31314 9910 18306 7545 9895 10584 18261
RELJCT2 int64 float64 int64 int64 int64

SCH_BUS 31330 129 129 279 0 150 150
SCH_BUS int64 float64 int64 int64 int64

URBANICITY 31232 7776 7773 7938 7 7854 7861
URBANICITY int64 float64 int64 int64 int64

VE_TOTAL 31260 6491 8449 1812 2102 6690 8512
VE_TOTAL int64 float64 int64 int64 int64

WEATHER 31372 8981 8981 10678 0 8823 8823
WEATH

,Feature,nNaN,nRF Incorrect,pRF Incorrect,nMode Incorrect,pMode Incorrect,nIVEware Incorrect,pIVEware Incorrect,RF and Mode Different,RF v/s Mode %,RF and IVEware Different,RF v/s IVEware %,Mode and IVEware Different,Mode v/s IVEware %
0,SCH_BUS,31330,129,0,129,0,279,0,0,0,150,0,150,0
1,BUS_USE,31356,113,0,113,0,54,0,0,0,106,0,106,0
2,DR_PRES,31319,8,0,8,0,17,0,0,0,9,0,9,0
3,EMER_USE,31377,95,0,95,0,115,0,0,0,92,0,92,0
4,FIRE_EXP,31309,41,0,41,0,80,0,0,0,39,0,39,0
5,HAZ_CNO,31282,5,0,5,0,2,0,0,0,5,0,5,0
6,HAZ_INV,31322,2,0,4,0,10,0,2,0,8,0,6,0
7,HAZ_PLAC,31349,8,0,10,0,10,0,2,0,2,0,0,0
8,HAZ_REL,31329,9,0,9,0,31325,99,0,0,31327,99,31327,99
9,HIT_RUN,31361,161,0,161,0,322,1,0,0,165,0,165,0



Error RF =  22.17
Error Mode =  28.42
Error IVEware =  23.84
nRF > nMode:  -273273
nRF > nIVEware:  -772
nModel > nIVEware:  -606673
Compare RF to Mode:  46 31 1
Compare RF to IVEware:  49 0 29
Compare Mode to IVEware:  36 1 41

Number of NaN in data_NaN:  2443202
RF Different from Mode:  273351
RF Different from IVEware:  606751
Mode Different from IVEware:  738833


In [ ]:
def Main():
    data = Get_Data()
    
#    data_Imputed = Impute_Full(data)
    data_Imputed = Impute_Round_Robin(data)
    data_Imputed.to_csv('../../Big_Files/CRSS_Imputed_All_12_22_22.csv')
#    display(data_Imputed.head(50))
    
    Check(data, data_Imputed)
    display(Audio(sound_file, autoplay=True))
    return 0
Main()